# Querying OpenTripPlanner

In [37]:
import requests
import numpy as np
import pandas as pd

## Adresser
Adresser kan hentes via api https://api.dataforsyningen.dk/adresser?kommunekode=0360&format=csv

In [20]:
# Function to fetch data from the API
def fetch_addresses(municipality_code, file_name="addresses.csv"):
    # API endpoint
    url = f"https://api.dataforsyningen.dk/adresser?kommunekode={municipality_code}&format=csv"
    
    # Make the API request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Save the content to a CSV file
        with open(file_name, 'wb') as file:
            file.write(response.content)
        print(f"Data successfully fetched and saved to {file_name}")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

# Function to load the CSV into a DataFrame
def load_addresses(file_name="addresses.csv"):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_name)
    print(f"Data successfully loaded into DataFrame with {len(df)} rows and {len(df.columns)} columns.")
    return df
    

In [31]:
# hent adresser for lolland
municipality_code = "0360"  # Example municipality code, you can change this
file_name = "addresses.csv"  # Example file name, you can change this
target_file_naem = "skoler.csv"

# Fetch the data from the API and save it to a CSV file
#fetch_addresses(municipality_code, file_name)

# Load the data into a DataFrame
adr_df = load_addresses(file_name)
target_df = pd.read_csv(target_file_naem)

Data successfully loaded into DataFrame with 37019 rows and 83 columns.


In [25]:
df.columns

Index(['id', 'status', 'oprettet', 'ændret', 'vejkode', 'vejnavn',
       'adresseringsvejnavn', 'husnr', 'etage', 'dør', 'supplerendebynavn',
       'postnr', 'postnrnavn', 'stormodtagerpostnr', 'stormodtagerpostnrnavn',
       'kommunekode', 'kommunenavn', 'ejerlavkode', 'ejerlavnavn',
       'matrikelnr', 'esrejendomsnr', 'etrs89koordinat_øst',
       'etrs89koordinat_nord', 'wgs84koordinat_bredde',
       'wgs84koordinat_længde', 'nøjagtighed', 'kilde', 'tekniskstandard',
       'tekstretning', 'ddkn_m100', 'ddkn_km1', 'ddkn_km10',
       'adressepunktændringsdato', 'adgangsadresseid', 'adgangsadresse_status',
       'adgangsadresse_oprettet', 'adgangsadresse_ændret', 'regionskode',
       'regionsnavn', 'jordstykke_ejerlavnavn', 'kvhx', 'sognekode',
       'sognenavn', 'politikredskode', 'politikredsnavn', 'retskredskode',
       'retskredsnavn', 'opstillingskredskode', 'opstillingskredsnavn', 'zone',
       'jordstykke_ejerlavkode', 'jordstykke_matrikelnr',
       'jordstykke_esr

In [29]:
adr_df[['vejpunkt_x', 'vejpunkt_y']].head(10)

,vejpunkt_x,vejpunkt_y
0,11.132708,54.834179
1,11.135409,54.835587
2,11.133407,54.836483
3,11.132408,54.835670
4,11.133116,54.833892
5,11.132953,54.834027
6,11.132405,54.834540
7,11.132852,54.834083
8,11.134409,54.836040
9,11.132802,54.834112


In [32]:
target_df

,navn,lon,lat
0,skole nr 1,11.393348,54.693586
1,nakskov skole,11.150079,54.839289


## OTP api

There are two api models to consider:
- The GTFS GraphQL API 
- Transmodel GraphQL API

##  The GTFS GraphQL API 
The documentation can be found at https://docs.opentripplanner.org/api/dev-2.x/graphql-gtfs/queries/plan
especialy usfull parameters are 
- arriveBy (Boolean): Whether the itinerary should depart at the specified time (false), or arrive to the destination at the specified time (true). Default value: false

In [16]:
# Define the endpoint URL
url = "http://localhost:8080/otp/gtfs/v1"

In [17]:

# Define the GraphQL query

query = """
{
  plan(
    from: {lat: 54.81018, lon: 11.21086}
    to: {lat: 54.83214, lon: 11.13550}
    date: "2024-08-22"
    time: "10:00"
    walkSpeed: 1.0
    arriveBy: false
    numItineraries: 1
  ) {
    itineraries {
      startTime
      duration
      walkDistance
      legs {
        mode
        distance
        route {
          shortName
        }
      }
    }
  }
}
"""


# Send the request
response = requests.post(url, json={'query': query})

# Check if the request was successful
if response.status_code == 200:
    # Parse and print the JSON response
    data = response.json()
    print(data)
else:
    # Print the error
    print(f"Query failed with status code {response.status_code}: {response.text}")

{'data': {'plan': {'itineraries': [{'startTime': 1724314563000, 'duration': 2455, 'walkDistance': 2803.79, 'legs': [{'mode': 'WALK', 'distance': 2528.9, 'route': None}, {'mode': 'RAIL', 'distance': 5052.36, 'route': {'shortName': '710R'}}, {'mode': 'WALK', 'distance': 274.89, 'route': None}]}]}}}


In [33]:
def get_travel_info(from_lat, from_lon, to_lat, to_lon, date, time, walk_speed=1.3):
    query = """
    {
      plan(
        from: {lat: %s, lon: %s}
        to: {lat: %s, lon: %s}
        date: "%s"
        time: "%s"
        walkSpeed: %f
        arriveBy: false
        numItineraries: 1
      ) {
        itineraries {
          startTime
          duration
          walkDistance
        }
      }
    }
    """ % (from_lat, from_lon, to_lat, to_lon, date, time, walk_speed)
    
    response = requests.post(url, json={'query': query})
    return response.json()

In [46]:
%%time
# Define date and time variables
date = "2024-08-22"
time = "10:00"

# Create an empty list to store results
results = []

# Iterate over the first 10 addresses and all targets
for _, adr_row in adr_df.iterrows():
    for _, target_row in target_df.iterrows():
        # Extract coordinates and name
        from_lat = adr_row['vejpunkt_y']
        from_lon = adr_row['vejpunkt_x']
        to_lat = target_row['lat']
        to_lon = target_row['lon']
        target_name = target_row['navn']
        
        # Get travel information
        travel_info = get_travel_info(from_lat, from_lon, to_lat, to_lon, date, time)
        
        # Extract required information from the response
        try:
            itinerary = travel_info['data']['plan']['itineraries'][0]
            start_time = itinerary['startTime']
            duration = itinerary['duration']
            walk_distance = itinerary['walkDistance']
            
        except (KeyError, IndexError):
            # Handle cases where no itinerary was found by setting placeholders
            start_time = np.nan
            duration = np.nan
            walk_distance = np.nan
            
        # Append the results
        results.append({
            'from_lat': from_lat,
            'from_lon': from_lon,
            'target_name': target_name,
            'startTime': start_time,
            'duration': duration,
            'walkDistance': walk_distance
        })

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.9 µs


In [47]:
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Display the first few rows of the resulting DataFrame
results_df.head()


,from_lat,from_lon,target_name,startTime,duration,walkDistance
0,54.834179,11.132708,skole nr 1,1.724315e+12,3628.0,1086.21
1,54.834179,11.132708,nakskov skole,1.724314e+12,1441.0,1788.71
2,54.835587,11.135409,skole nr 1,1.724315e+12,3464.0,845.68
3,54.835587,11.135409,nakskov skole,1.724314e+12,1202.0,1488.58
4,54.836483,11.133407,skole nr 1,1.724315e+12,3571.0,969.80


In [48]:
# Specify the file name
output_file_name = "alle_adr_results.csv"

# Save the filtered DataFrame to a CSV file
results_df.to_csv(output_file_name, index=False)

# Confirm the file was saved
print(f"Filtered results saved to {output_file_name}")

Filtered results saved to alle_adr_results.csv


In [43]:
# Function to filter rows for each group
def filter_min_duration(group):
    # Check if all durations are NaN
    if group['duration'].isna().all():
        # If all durations are NaN, return one row (could be the first row)
        return group.head(1)
    else:
        # Otherwise, return the row with the minimum duration
        return group.loc[group['duration'].idxmin()]

# Apply the filtering function to each group
filtered_df = results_df.groupby(['from_lat', 'from_lon'], group_keys=False).apply(filter_min_duration)

# Reset index if needed
filtered_df = filtered_df.reset_index(drop=True)

# Display the filtered DataFrame
filtered_df.head()

/var/folders/w3/1_ytt14d0w7cyn40yksjj4xm0000gn/T/ipykernel_94244/2163227587.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = results_df.groupby(['from_lat', 'from_lon'], group_keys=False).apply(filter_min_duration)


,10358,10360,10366,10350,10344,10352,10356,10362,10346,10364,...,3263,7637,3265,3267,10751,7635,10757,10749,7638,7640
0,54.634274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.417668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,skole nr 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1724313600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  Transmodel GraphQL API

In [49]:
# Define the endpoint URL
url = "http://localhost:8080/otp/transmodel/v3"

In [51]:

# Define the GraphQL query
query = """
{
  trip(
    from: {name: "start", coordinates: {latitude: 54.81018, longitude: 11.21086}}
    to: {name: "slut", coordinates: {latitude: 54.83214, longitude: 11.13550}}
    numTripPatterns: 3
    dateTime: "2024-08-23T13:08:00+02:00"
    walkSpeed: 1.3
    arriveBy: false
  ) {
    tripPatterns {
      expectedStartTime
      expectedEndTime
      duration
      walkDistance
      legs {
        mode
        distance
        line {
          id
          publicCode
        }
      }
    }
  }
}
"""


# Send the request
response = requests.post(url, json={'query': query})

# Check if the request was successful
if response.status_code == 200:
    # Parse and print the JSON response
    data = response.json()
    print(data)
else:
    # Print the error
    print(f"Query failed with status code {response.status_code}: {response.text}")

{'data': {'trip': {'tripPatterns': [{'expectedStartTime': '2024-08-23T13:16:46+02:00', 'duration': 2412, 'walkDistance': 429.7, 'legs': [{'mode': 'foot', 'distance': 87.45, 'line': None}, {'mode': 'bus', 'distance': 4011.16, 'line': {'id': 'EN:DK::Line:20533::', 'publicCode': '717'}}, {'mode': 'foot', 'distance': 67.36, 'line': None}, {'mode': 'rail', 'distance': 9009.21, 'line': {'id': 'EN:DK::Line:24603::', 'publicCode': '710R'}}, {'mode': 'foot', 'distance': 274.89, 'line': None}]}]}}}
